# Disease Classification using Machine Learning

# Table of Contents

1. [Introduction](#Introduction)
    - [Overview of the Dataset](#Overview-of-the-Dataset)
    - [Objectives of the Analysis](#Objectives-of-the-Analysis)
2. [Data Loading](#Data-Loading)
    - [Importing Libraries](#Importing-Libraries)
    - [Loading the Dataset](#Loading-the-Dataset)
3. [Data Inspection](#Data-Inspection)
    - [Initial Data Exploration](#Initial-Data-Exploration)
    - [Overview of Dataset](#Overview-of-Dataset)
    - [Data Information](#Data-Information)
    - [Analyzing Prognosis Distribution for Potential Class Imbalance](#class-imbalance-analysis)
    - [Summary Statistics](#Summary-Statistics)
    - [Missing Values](#Missing-Values)
    - [Duplicate Records](#Duplicate-Records)
4. [Data Transformation](#Data-Transformation)
    - [Feature Engineering](#Feature-Engineering)
5. [Data Visualization](#Data-Visualization)
    - [Symptom Frequency Distribution](#Sympton-Frequency-Distribution)

# Introduction
<a id="Introduction"></a>

## Overview of the Dataset
<a id="Overview-of-the-Dataset"></a>
In this project, we aim to predict the disease by analyzing different symptoms a person experiences, this is captured in the last column name 'prognosis'. We have around 132 parameters on which 42 different types of diseases can be predicted. 


## Objectives of the Analysis
<a id="Objectives-of-the-Analysis"></a>
The primary objective of this project is to develop a machine-learning model that can accurately classify and cluster diseases based on patient's symptoms. This can be used to a helping tool for medical staff to help patients all around the world. Best features should be identifies while creating of the ML model.

### Data Loading


##### import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.colors as mcolors
from mpl_toolkits import mplot3d 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split 
import missingno as msno

In [ ]:
def read_train_data (train_file_path):
    # Load data and convert to pd
    train = pd.read_csv(train_file_path)
    return train

def read_test_data (test_file_path):
    # Load data and convert to pd
    test = pd.read_csv(test_file_path)
    return test

##### Loading the Dataset


In [ ]:
train_data = read_train_data("data/Training.csv")
test_data = read_test_data("data/Testing.csv")

# Data Inspection
<a id="Data-Inspection"></a>
## Initial Data Exploration
<a id="Initial-Data-Exploration"></a>


## Overview of Dataset
<a id="Overview-of-Dataset"></a>

## Data Information
<a id="Data-Information"></a>

## Analyzing Prognosis Distribution for Potential Class Imbalance
<a id="class-imbalance-analysis"></a>

## Summary Statistics
<a id="Summary-Statistics"></a>


## Missing Values
<a id="Missing-Values"></a>

# Data Transformation
<a id="Data-Transformation"></a>
## Feature Engineering
<a id="Feature-Engineering"></a>


# Data Visualization
<a id="Data-Visualization"></a>
## Symptom Frequency Distribution
<a id="Sympton-Frequency-Distribution"></a>